### Dataset de companies

In [86]:
from pymongo import MongoClient
import pandas as pd

In [87]:
conn = MongoClient("localhost:27017")

In [88]:
conn.list_database_names()

['admin', 'config', 'ironhack', 'local']

In [89]:
db = conn.get_database("ironhack")

In [90]:
db.list_collection_names()

['countries_small', 'companies', 'restaurants']

In [91]:
collection = db.get_collection("companies")

In [92]:
data = collection.find()

In [93]:
collection.find_one()


{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
 'name': 'AdventNet',
 'permalink': 'abc3',
 'crunchbase_url': 'http://www.crunchbase.com/company/adventnet',
 'homepage_url': 'http://adventnet.com',
 'blog_url': '',
 'blog_feed_url': '',
 'twitter_username': 'manageengine',
 'category_code': 'enterprise',
 'number_of_employees': 600,
 'founded_year': 1996,
 'deadpooled_year': 2,
 'tag_list': '',
 'alias_list': 'Zoho ManageEngine ',
 'email_address': 'pr@adventnet.com',
 'phone_number': '925-924-9500',
 'description': 'Server Management Software',
 'created_at': datetime.datetime(2007, 5, 25, 19, 24, 22),
 'updated_at': 'Wed Oct 31 18:26:09 UTC 2012',
 'overview': '<p>AdventNet is now <a href="/company/zoho-manageengine" title="Zoho ManageEngine" rel="nofollow">Zoho ManageEngine</a>.</p>\n\n<p>Founded in 1996, AdventNet has served a diverse range of enterprise IT, networking and telecom customers.</p>\n\n<p>AdventNet supplies server and network management software.</p>',
 'image': {'avail

### Decido que a mi empresa le interesa estar cerca de empresas que facturen mas de 1 millón de euros.

In [99]:
companies = collection.find({'total_money_raised': {"$regex": ".*\M$"}}, {"offices": 1, 'name': 1, '_id': 0})
# utilizo regex para filtrarlo porque me doy cuenta que siempre que supera el millón de euros acaba por M

#### Paso el diccionario a lista

In [95]:
lst_companies = list(companies)

In [96]:
len(lst_companies)

4030

In [53]:
lst_companies[0]

{'name': 'Wetpaint',
 'offices': [{'description': '',
   'address1': '710 - 2nd Avenue',
   'address2': 'Suite 1100',
   'zip_code': '98104',
   'city': 'Seattle',
   'state_code': 'WA',
   'country_code': 'USA',
   'latitude': 47.603122,
   'longitude': -122.333253},
  {'description': '',
   'address1': '270 Lafayette Street',
   'address2': 'Suite 505',
   'zip_code': '10012',
   'city': 'New York',
   'state_code': 'NY',
   'country_code': 'USA',
   'latitude': 40.7237306,
   'longitude': -73.9964312}]}

In [54]:
### Hago un bucle para que me itere por todos los elementos de la lista y para cada elemento me entre en el valor de offices
lst_offices =[]
for i in range(0, len(lst_companies)):
    lst_companies[i].get('offices')
    lst_offices.append(lst_companies[i].get('offices'))

lst_offices    

[[{'description': '',
   'address1': '710 - 2nd Avenue',
   'address2': 'Suite 1100',
   'zip_code': '98104',
   'city': 'Seattle',
   'state_code': 'WA',
   'country_code': 'USA',
   'latitude': 47.603122,
   'longitude': -122.333253},
  {'description': '',
   'address1': '270 Lafayette Street',
   'address2': 'Suite 505',
   'zip_code': '10012',
   'city': 'New York',
   'state_code': 'NY',
   'country_code': 'USA',
   'latitude': 40.7237306,
   'longitude': -73.9964312}],
 [{'description': 'Headquarters',
   'address1': '9229 W. Sunset Blvd.',
   'address2': '',
   'zip_code': '90069',
   'city': 'West Hollywood',
   'state_code': 'CA',
   'country_code': 'USA',
   'latitude': 34.090368,
   'longitude': -118.393064}],
 [{'description': None,
   'address1': None,
   'address2': None,
   'zip_code': None,
   'city': 'Menlo Park',
   'state_code': 'CA',
   'country_code': 'USA',
   'latitude': 37.48413,
   'longitude': -122.169472}],
 [{'description': '',
   'address1': '',
   'address

### Creo un DataFrame a partir de la lista de diccionarios

In [69]:
from functools import reduce
import operator

In [74]:
city = ['city']
country_code = ["country_code"]
latitud = ["latitude"]
longitud = ["longitude"]

In [75]:
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)


In [76]:
lst_1 = []
for lst in lst_offices:  ### para entrar a cada diccionario tengo que hacer un doble forllop
    for diccionario in lst:
        paralista = {}
        paralista['city'] = getFromDict(diccionario,city)
        paralista['country_code'] = getFromDict(diccionario,country_code)
        paralista['latitud'] = getFromDict(diccionario,latitud)
        paralista['longitud'] = getFromDict(diccionario,longitud)
        lst_1.append(paralista)

In [77]:
dataframefinal = pd.DataFrame(lst_1)
dataframefinal

,city,country_code,latitud,longitud
0,Seattle,USA,47.603122,-122.333253
1,New York,USA,40.723731,-73.996431
2,West Hollywood,USA,34.090368,-118.393064
3,Menlo Park,USA,37.484130,-122.169472
4,San Francisco,USA,37.775196,-122.419204
...,...,...,...,...
5191,Orlando,USA,NaN,NaN
5192,Portland,USA,NaN,NaN
5193,London,GBR,NaN,NaN
5194,Brussels,BEL,50.839970,4.346472


In [60]:
df = dataframefinal[dataframefinal.country_code =='USA'] ## filtro solo para las empresas de USA, porque quiero que mi empresa este situada allí

In [61]:
df

,city,country_code,latitud,longitud
0,Seattle,USA,47.603122,-122.333253
1,New York,USA,40.723731,-73.996431
2,West Hollywood,USA,34.090368,-118.393064
3,Menlo Park,USA,37.484130,-122.169472
4,San Francisco,USA,37.775196,-122.419204
...,...,...,...,...
5189,San Jose,USA,37.386417,-121.929170
5190,Newark,USA,37.512038,-121.998513
5191,Orlando,USA,NaN,NaN
5192,Portland,USA,NaN,NaN


In [62]:
df_1 = df.dropna() # Borro todas las filas que tengan Nan para poder hacer luego el mapa y porque si no tienen latitud ni longitud no me interesan

In [63]:
df_1

,city,country_code,latitud,longitud
0,Seattle,USA,47.603122,-122.333253
1,New York,USA,40.723731,-73.996431
2,West Hollywood,USA,34.090368,-118.393064
3,Menlo Park,USA,37.484130,-122.169472
4,San Francisco,USA,37.775196,-122.419204
...,...,...,...,...
5186,San Clemente,USA,33.448010,-117.607717
5188,Menlo Park,USA,37.484616,-122.181753
5189,San Jose,USA,37.386417,-121.929170
5190,Newark,USA,37.512038,-121.998513


In [80]:
df_1.city.value_counts() # Elijo Palo Alto porque es una zona con muchas empresas y ademas es una buena zona residencial

San Francisco    317
New York         219
Mountain View     90
Seattle           88
Palo Alto         84
                ... 
Gainesville        1
Florham Park       1
Branchburg         1
Belcamp            1
Arcadia            1
Name: city, Length: 466, dtype: int64

In [81]:
df_1['type']= 'empresa' # Creo la columna type para más adelante poder concatenar los DataFrames y diferenciar los markers 

<ipython-input-81-327dd6e2b927>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['type']= 'empresa'


In [82]:
df_1

,city,country_code,latitud,longitud,type
0,Seattle,USA,47.603122,-122.333253,empresa
1,New York,USA,40.723731,-73.996431,empresa
2,West Hollywood,USA,34.090368,-118.393064,empresa
3,Menlo Park,USA,37.484130,-122.169472,empresa
4,San Francisco,USA,37.775196,-122.419204,empresa
...,...,...,...,...,...
5186,San Clemente,USA,33.448010,-117.607717,empresa
5188,Menlo Park,USA,37.484616,-122.181753,empresa
5189,San Jose,USA,37.386417,-121.929170,empresa
5190,Newark,USA,37.512038,-121.998513,empresa


In [29]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd

In [30]:
map_1 = folium.Map(location = [47.603122,-122.333253], zoom_start = 15)

In [31]:
map_1

In [97]:
for i,row in df_1.iterrows():
    ubicación = {
        "location" : [row["latitud"], row["longitud"]],
    }
    Marker (**ubicación).add_to(map_1)

In [98]:
map_1

In [41]:
map_1.save('Data/mapa_empresas')

In [83]:
df_1.to_csv('empresas.csv')